In [2]:
import trimesh
import vedo
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from IPython.display import display, HTML

class Visualizador3D:
    def __init__(self, ruta_modelo):
        """Inicializa el visualizador con la ruta del modelo 3D"""
        self.ruta_modelo = ruta_modelo
        self.extension = os.path.splitext(ruta_modelo)[1].lower()
        
        # Cargar el modelo según su extensión
        if self.extension in ['.obj', '.stl', '.gltf', '.glb']:
            self.mesh_trimesh = trimesh.load(ruta_modelo)
            self.mesh_vedo = vedo.Mesh(ruta_modelo)
        else:
            raise ValueError(f"Formato de archivo no soportado: {self.extension}")
        
    def mostrar_informacion(self):
        """Muestra información estructural del modelo"""
        info = {
            "Número de vértices": len(self.mesh_trimesh.vertices),
            "Número de caras": len(self.mesh_trimesh.faces),
            "Número de aristas": len(self.mesh_trimesh.edges),
            "Tamaño (mm)": self.mesh_trimesh.bounding_box.extents,
            "Volumen (mm³)": self.mesh_trimesh.volume,
            "Es hermético": self.mesh_trimesh.is_watertight,
            "Centro de masa": self.mesh_trimesh.center_mass
        }
        
        print("\n=== INFORMACIÓN DEL MODELO ===")
        for clave, valor in info.items():
            print(f"{clave}: {valor}")
        return info
    
    def visualizar_con_vedo(self):
        """Visualiza el modelo con vedo, mostrando vértices, aristas y caras"""
        # Creamos una escena
        plt = vedo.Plotter(title=f"Modelo: {os.path.basename(self.ruta_modelo)}")
        
        # Preparamos las representaciones visuales con colores distintos
        vertices = vedo.Points(self.mesh_vedo.points, r=8, c="magenta")
        aristas = self.mesh_vedo.wireframe().color("cyan")
        caras = self.mesh_vedo.clone().color("yellow").alpha(0.7)
        
        # Añadimos las representaciones a la escena
        plt.add(vertices, "Vértices")
        plt.add(aristas, "Aristas")
        plt.add(caras, "Caras")
        
        # Mostramos la escena
        plt.show()
        
    def generar_animacion(self, duracion=5, fps=30, ruta_salida=None):
        """Genera una animación rotando el modelo y la guarda como GIF"""
        if ruta_salida is None:
            nombre_base = os.path.splitext(os.path.basename(self.ruta_modelo))[0]
            ruta_salida = f"{nombre_base}_animacion.gif"
        
        # Configuración de la figura
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111, projection='3d')
        
        # Obtener vértices y caras del modelo
        vertices = self.mesh_trimesh.vertices
        caras = self.mesh_trimesh.faces
        
        # Normalizar vértices para centrar el modelo
        vertices = vertices - np.mean(vertices, axis=0)
        
        # Función para actualizar la animación
        def actualizar(frame):
            ax.clear()
            
            # Rotar los vértices
            angulo = frame * 2 * np.pi / (fps * duracion)
            
            # Matriz de rotación alrededor del eje Y (para que gire horizontalmente)
            matriz_rotacion = np.array([
                [np.cos(angulo), 0, np.sin(angulo)],
                [0, 1, 0],
                [-np.sin(angulo), 0, np.cos(angulo)]
            ])
            
            vertices_rotados = np.dot(vertices, matriz_rotacion)
            
            # Aplicar una rotación inicial para corregir la orientación
            # Rotación de -90 grados alrededor del eje X para corregir la orientación
            # Cambiamos el ángulo de corrección para que el modelo se muestre "de pie"
            angulo_correccion = -np.pi/2
            matriz_correccion = np.array([
                [1, 0, 0],
                [0, np.cos(angulo_correccion), -np.sin(angulo_correccion)],
                [0, np.sin(angulo_correccion), np.cos(angulo_correccion)]
            ])
            
            vertices_rotados = np.dot(vertices_rotados, matriz_correccion)
            
            # Dibujar el modelo
            ax.set_xlim([-1, 1])
            ax.set_ylim([-1, 1])
            ax.set_zlim([-1, 1])
            ax.set_axis_off()
            
            # Dibujar caras con un color diferente
            ax.plot_trisurf(
                vertices_rotados[:, 0], 
                vertices_rotados[:, 1], 
                vertices_rotados[:, 2],
                triangles=caras,
                alpha=0.7,
                color='orange'
            )
            
            # Dibujar aristas
            for cara in caras:
                for i in range(3):
                    idx1, idx2 = cara[i], cara[(i+1)%3]
                    ax.plot(
                        [vertices_rotados[idx1, 0], vertices_rotados[idx2, 0]],
                        [vertices_rotados[idx1, 1], vertices_rotados[idx2, 1]],
                        [vertices_rotados[idx1, 2], vertices_rotados[idx2, 2]],
                        color='purple', linewidth=1
                    )
            
            # Dibujar vértices
            ax.scatter(
                vertices_rotados[:, 0],
                vertices_rotados[:, 1],
                vertices_rotados[:, 2],
                color='lime', s=10
            )
            
            return ax,
        
        # Crear la animación
        frames_totales = fps * duracion
        anim = FuncAnimation(
            fig, 
            actualizar, 
            frames=frames_totales, 
            interval=1000/fps, 
            blit=False
        )
        
        # Guardar como GIF
        anim.save(ruta_salida, writer='pillow', fps=fps)
        print(f"Animación guardada como: {ruta_salida}")
        
        plt.close(fig)
        return ruta_salida

# Ejemplo de uso
ruta_modelo = "C:\\Users\\USUARIO\\Downloads\\silla_simple.obj" 

# Para usar el visualizador, descomenta las siguientes líneas y ajusta la ruta
visualizador = Visualizador3D(ruta_modelo)
visualizador.mostrar_informacion()
visualizador.visualizar_con_vedo()
visualizador.generar_animacion(duracion=5, fps=30)



=== INFORMACIÓN DEL MODELO ===
Número de vértices: 42
Número de caras: 72
Número de aristas: 216
Tamaño (mm): [0.4  0.85 0.4 ]
Volumen (mm³): 0.04800000000000002
Es hermético: False
Centro de masa: [0.18541667 0.19583333 0.62291667]
Animación guardada como: silla_simple_animacion.gif


'silla_simple_animacion.gif'